# Programming language interoperability

## Python

In [1]:
using PyCall

In [2]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/crstnbr/opt/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>

In [3]:
np.linalg.eigvals(np.random.rand(5,5))

5-element Array{Complex{Float64},1}:
   2.7023702983746003 + 0.0im
 -0.36250324628611025 + 0.3673021141132176im
 -0.36250324628611025 - 0.3673021141132176im
 0.041843202490917795 + 0.0im
    0.587676783916915 + 0.0im

In [4]:
M = rand(5,5)
np.linalg.eigvals(M)

5-element Array{Complex{Float64},1}:
  2.9209184695697035 + 0.0im
 -0.4789533391427205 + 0.0im
 -0.2032084007238829 + 0.24969874362559003im
 -0.2032084007238829 - 0.24969874362559003im
   0.666037846588996 + 0.0im

In [6]:
py"""
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [7]:
py_sinpi(x) = py"sinpi"(x)

py_sinpi (generic function with 1 method)

In [8]:
py_sinpi(10)

-1.2246467991473533e-15

In [9]:
using BenchmarkTools
@btime py_sinpi(10);

  10.161 μs (10 allocations: 256 bytes)


## C

In [10]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [11]:
c_code

"#include <stddef.h>\ndouble c_sum(size_t n, double *X) {\n    double s = 0.0;\n    for (size_t i = 0; i < n; ++i) {\n        s += X[i];\n    }\n    return s;\n}\n"

Compile to a shared library by piping `c_code` to gcc:

In [12]:
using Libdl
const Clib = tempname()

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

Binding the function from the shared library:

In [13]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [14]:
c_sum(rand(10))

5.228411516051957

In [15]:
x = rand(10)
@btime c_sum($x);

  3.650 ns (0 allocations: 0 bytes)


## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [16]:
x = rand(10);

In [17]:
py_sinpi(real(c_sum(x)))

0.7462204552500249

In [18]:
@btime py_sinpi(real(c_sum($x)));

  9.963 μs (8 allocations: 224 bytes)


See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).